In [29]:
%load_ext sql

In [30]:
%config SqlMagic.displaylimit = 15
# %config SqlMagic.displaylimit = None

In [ ]:
import datetime as dt
import sqlite3
import pandas as pd
from sqlalchemy import create_engine, text

In [42]:
sys_log_db = "system_data_log.db"

sql_engine = create_engine("sqlite:///%s" % sys_log_db, echo=False)

In [189]:
def create_voltage_table(force=False):
    with sql_engine.connect() as sql_conn:
        if force:
            sql_text = text("""DROP TABLE IF EXISTS voltages;
                            """)
            sql_conn.execute(sql_text)
        # define schema
        sql_text = text("""
            CREATE TABLE IF NOT EXISTS voltages (
                Timestamp TEXT,
                Vmain FLOAT,
                Vmain_raw FLOAT,
                Vaux FLOAT,
                Vaux_raw FLOAT,
                PRIMARY KEY (Timestamp)
                )
        """)
        sql_conn.execute(sql_text)
        sql_conn.commit()

In [ ]:
create_voltage_table()

In [193]:
def add_voltage_log(Vmain, Vmain_raw, Vaux, Vaux_raw):
    with sql_engine.connect() as sql_conn:
        sql_text = text("""INSERT INTO voltages
                           VALUES (DateTime("now", "localtime"),
                                   :v_main,
                                   :v_main_raw,
                                   :v_aux,
                                   :v_aux_raw
                           );
                        """)
        sql_conn.execute(sql_text, {"v_main": Vmain,
                                    "v_main_raw": Vmain_raw,
                                    "v_aux": Vaux,
                                    "v_aux_raw": Vaux_raw})
        sql_conn.commit()

In [194]:
add_voltage_log(12.4, 12.8, 12.5, 12)

In [195]:
add_voltage_log(12.5, 12.75, 12.4, 11.9)

---

In [282]:
def get_voltages(voltage_type="Vmain", trailing_seconds=None):
    with sql_engine.connect() as sql_conn:
        sql_text = text(f"""SELECT Timestamp, {voltage_type}
                            FROM voltages
                         """)
        result_df = pd.read_sql(sql_text, con=sql_conn, index_col = "Timestamp", parse_dates=["Timestamp"])
    if trailing_seconds is not None:
        return result_df[result_df.index > dt.datetime.now() - dt.timedelta(seconds=trailing_seconds)]
    else:
        return result_df

In [343]:
def get_voltages(voltage_type=None, trailing_seconds=None):
    if voltage_type is None:
        cols = "*"
    else:
        cols = "Timestamp, %s" % voltage_type

    if trailing_seconds is not None:
        timestamp_now = dt.datetime.now()
        timestamp_trail = timestamp_now - dt.timedelta(seconds=trailing_seconds)
        timestamp_trail_str = timestamp_trail.strftime("%Y-%m-%d %H:%M:%S")
        time_filter = "WHERE Timestamp > %s" % timestamp_trail_str
    else:
        time_filter = ""

    with sql_engine.connect() as sql_conn:
        # sql_text = text(f"""SELECT Timestamp, {voltage_type}
        sql_text = text(f"""SELECT {cols}
                            FROM voltages
                            {time_filter}
                         """)
        result_df = pd.read_sql(sql_text, con=sql_conn, index_col = "Timestamp", parse_dates=["Timestamp"])
    return result_df

In [344]:
df = get_voltages("Vmain"); df

,Vmain
Timestamp,
2025-10-12 15:08:14,12.4
2025-10-12 15:08:21,12.4
2025-10-12 15:08:22,12.5


---

In [529]:
from importlib import reload
import class_def
reload(class_def)
dl = class_def.DataLogger()

In [ ]:
dl.get_voltages(dt.datetime.now(), column_list=["Vmain_raw"])

,Vmain_raw
Timestamp,


In [510]:
dl.get_charging(dt.datetime.now())

,charge_enable,charge_dir,charge_current,shunt_V_in,shunt_V_out
Timestamp,,,,,
2025-10-13 21:16:05,1,1,4.3,0.1,0.03
2025-10-13 21:16:45,1,1,4.3,0.1,0.03
2025-10-13 21:17:03,1,1,4.3,0.1,0.03
2025-10-13 21:17:50,1,1,4.3,0.1,0.03
2025-10-13 21:27:45,1,1,4.3,0.1,0.03
2025-10-13 21:28:15,1,1,4.3,0.1,0.03
2025-10-13 21:30:07,1,1,4.3,0.1,0.03


In [442]:
dl.log_charging(dt.datetime.now(), [True, True, 4.3, 0.1, 0.03])

In [464]:
dl.log_signals(dt.datetime.now(), [True, False, False, False, "network 5", 13.2, 13, 12.5, True, False, False, True, True, False])

In [433]:
str(None).replace("None", "Null")

'Null'

---

In [479]:
dl.get_signals(dt.datetime.now())

,enable_sw,key_ACC,ecu_W,engine_on,network_conn,HAT_analog_0,HAT_analog_1,HAT_analog_2,HAT_input_0,HAT_input_1,HAT_input_2,HAT_relay_0,HAT_relay_1,HAT_relay_2
Timestamp,,,,,,,,,,,,,,
2025-10-13 21:23:54,1,0,0,0,None,13.2,13.0,12.5,1,0,0,1,1,0
2025-10-13 21:28:13,1,0,0,0,None,13.2,13.0,12.5,1,0,0,1,1,0
2025-10-13 21:30:07,1,0,0,0,None,13.2,13.0,12.5,1,0,0,1,1,0
2025-10-13 21:39:18,1,0,0,0,NETWORK 5,13.2,13.0,12.5,1,0,0,1,1,0
2025-10-17 14:52:50,1,0,0,0,network 5,13.2,13.0,12.5,1,0,0,1,1,0
2025-10-17 14:55:58,1,0,0,0,phone hotspot,13.2,13.0,12.5,1,0,0,1,1,0
2025-10-17 14:56:20,1,0,0,0,None,13.2,13.0,12.5,1,0,0,1,1,0


In [371]:
dl.get_charging(dt.datetime.now())

,charge_enable,charge_dir,charge_current,shunt_V_in,shunt_V_out
Timestamp,,,,,
2025-10-13 21:16:05,1,1,4.3,0.1,0.03
2025-10-13 21:16:45,1,1,4.3,0.1,0.03
2025-10-13 21:17:03,1,1,4.3,0.1,0.03
2025-10-13 21:17:50,1,1,4.3,0.1,0.03


---

In [470]:
", ".join([str(x).upper() if not isinstance(x, str) else "'%s'" % x for x in values_list]).replace("NONE", "NULL") 

"TRUE, FALSE, FALSE, FALSE, 'phone hotspot', 13.2, 13, 12.5, TRUE, FALSE, FALSE, TRUE, TRUE, FALSE"

In [455]:
# values_list = [True, False, False, False, "network 5", 13.2, 13, 12.5, True, False, False, True, True, False]
values_list = [True, False, False, False, None, 13.2, 13, 12.5, True, False, False, True, True, False]

In [474]:
import subprocess
from network_names import stored_ssid_mapping_dict     # local file
# stored_ssid_mapping_dict.get(subprocess.run(["/usr/sbin/iwgetid", "-r"], capture_output=True, text=True).stdout.strip()),

values_list = [True, False, False, False, 
               stored_ssid_mapping_dict.get(subprocess.run(["/usr/sbin/iwgetid", "-r"], capture_output=True, text=True).stdout.strip()),
               13.2, 13, 12.5, True, False, False, True, True, False]

dl.log_signals(dt.datetime.now(), values_list)

---

In [ ]:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_sql.html
pandas.DataFrame.to_sql(sys_log_db, )

In [481]:
def fun1(l1, l2, l3):
    print(l1)
    print(l2)
    print(l3)
fun1(*[0, 1, 2])

0
1
2


In [486]:
[x for x in range(0, 2+1)]

[0, 1, 2]

In [487]:
table_df = dl.get_charging(dt.datetime.now()); table_df

,charge_enable,charge_dir,charge_current,shunt_V_in,shunt_V_out
Timestamp,,,,,
2025-10-13 21:16:05,1,1,4.3,0.1,0.03
2025-10-13 21:16:45,1,1,4.3,0.1,0.03
2025-10-13 21:17:03,1,1,4.3,0.1,0.03
2025-10-13 21:17:50,1,1,4.3,0.1,0.03
2025-10-13 21:27:45,1,1,4.3,0.1,0.03
2025-10-13 21:28:15,1,1,4.3,0.1,0.03
2025-10-13 21:30:07,1,1,4.3,0.1,0.03


In [493]:
time_now = dt.datetime.now()

In [497]:
(time_now - table_df.index[-1]) > dt.timedelta(seconds=5)

True

In [498]:
table_df = dl.get_charging(dt.datetime.now(), 5); table_df

,charge_enable,charge_dir,charge_current,shunt_V_in,shunt_V_out
Timestamp,,,,,


In [499]:
len(table_df.index)

0